Shell for the functions needed for the gpt model

In [78]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
from tqdm.auto import tqdm

key = jax.random.PRNGKey(420)

# Hyperparameters
batch_size = 4
context_lenght = 8
train_test_split_size = 0.9

In [79]:
def open_data(path: str = "new_nietzsche.txt"):
    txt = open(path, "r", encoding="utf-8").read()
    return txt

text = open_data()

In [80]:
class Tokenizer:
    """
    Class that takes care of encoding and decoding the text
    """

    def __init__(self, text:str, tokenizer_type:str="base") -> None:
        self.tokenizer_type = tokenizer_type
        self.vocab_size, self.all_characters = self.sort_characters(text)


    def get_vocab_size(self):
        return jnp.copy(self.vocab_size)

    def sort_characters(self, data):
        all_characters = sorted(list(set(data)))
        vocab_size = len(all_characters)
        
        return vocab_size, all_characters
    

    def encode(self, text):
        encoded_text = []
        if self.tokenizer_type == "base":
            for c in text:
                num = self.all_characters.index(c)
                encoded_text.append(num)
        return jnp.array(encoded_text)

    def decode(self, encoded_text):
        text = []
        if self.tokenizer_type == "base":
            for n in encoded_text:
                char = self.all_characters[n]
                text.append(char)
            text = "".join([str(item) for item in text])

        return text

In [81]:
tokenizer = Tokenizer(text=text, tokenizer_type="base")
data = tokenizer.encode(text)
len(data)

3396780

In [82]:
# test tokenizer
print(tokenizer.decode(data[:500]))

What I am now going to relate is the history of the next two centuries.
I shall describe what will happen, what must necessarily happen:
the triumph of Nihilism. This history can be written already; for
necessity itself is at work in bringing it about. This future is
already proclaimed by a hundred different omens; as a destiny it
announces its advent everywhere, for this music of to-morrow all ears
are already pricked. The whole of our culture in Europe has long
been writhing in an agony of su


In [83]:
class BatchLoader:
    def __init__(self, data, train_test_split_size) -> None:
        self.training_data, self.validation_data = self.splitting_data(
            data, train_test_split_size
        )

    def splitting_data(self, data, split_size):
        n = int(split_size * len(data))
        training_data = data[:n]
        validation_data = data[n:]
        return training_data, validation_data

    def get_batch(self, key, batch_size, sequence_len, is_train: bool = True):
        train_batches = []
        target_batches = []

        if is_train:
            b_data = self.training_data
        else:
            b_data = self.validation_data

        for _ in range(batch_size):
            key, subkey = jax.random.split(key)
            pos = jax.random.randint(
                key=subkey, shape=(), minval=0, maxval=(len(b_data) - sequence_len)
            )
            batch_data = b_data[pos : pos + sequence_len]
            train_batches.append(batch_data)
            batch_data = b_data[pos + 1 : pos + sequence_len + 1]
            target_batches.append(batch_data)
            key = subkey

        train_batch = jnp.stack(train_batches)
        target_batch = jnp.stack(target_batches)

        return train_batch, target_batch

In [84]:
batch_loader = BatchLoader(data=data, train_test_split_size=train_test_split_size)
train_batch, target_batch = batch_loader.get_batch(
    key, batch_size, context_lenght, is_train=True
)
print(train_batch)
print(target_batch)

[[ 1 76 64 57 76  1 75 71]
 [70 28  1 37 76  1 65 75]
 [64  0 72 68 57 81 75  1]
 [61 68 57 76 65 71 70 75]]
[[76 64 57 76  1 75 71 69]
 [28  1 37 76  1 65 75  0]
 [ 0 72 68 57 81 75  1 76]
 [68 57 76 65 71 70 75  1]]


In [85]:
class BenchmarkModel(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(
            num_embeddings=self.vocab_size, features=self.vocab_size
        )

    def __call__(self, data, labels=None):
        logits = self.token_embedding_table(data)
        return logits

    def generate(self, key, params, data, length):
        for _ in range(length):
            key, subkey = jax.random.split(
                key
            )  # bcs every character has to be different

            logits = self.apply({"params": params}, data)

            logits = logits[:, -1, :]
            probabilities = jax.nn.softmax(logits)
            probabilities = jax.numpy.squeeze(probabilities)

            next_token = jax.random.choice(
                subkey, jax.numpy.arange(self.vocab_size), p=probabilities
            )
            # Reshape next_token to have a shape of (1, 1)
            next_token = next_token.reshape((1, 1))
            data = jax.numpy.concatenate((data, next_token), axis=1)

        return data

In [86]:
# Optimizer

scheduler = optax.warmup_cosine_decay_schedule(
    init_value=0.01, peak_value=1, warmup_steps=100, decay_steps=2000
)
optimizer = optax.adamw(scheduler)

In [87]:
# @jax.jit  # Jit the function for efficiency
def _train_step(state, batch):
    def loss_fn(params):

        data, labels = batch
        print(data.shape)
        # Same as model.apply
        logits = state.apply_fn(
            {"params": params},
            data,
        )

        b, t, c = logits.shape
        logits = logits.reshape((b * t, c))
        labels = labels.reshape((b * t))
        labels_one_hot = nn.one_hot(labels, num_classes=c)


        loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
        mean_loss = jnp.mean(loss)
        return mean_loss, logits

    # Gradient function
    grad_fn = jax.value_and_grad(
        loss_fn,  # Function to calculate the loss
        has_aux=True,  # Function has additional outputs, here accuracy
    )
    # Determine gradients for current model, parameters and batch
    (loss, logits), grads = grad_fn(state.params)
    # accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)

    # Perform parameter update with gradients and optimizer
    state = state.apply_gradients(grads=grads)
    # Return state and any other value we might want
    return state, loss


@jax.jit  # Jit the function for efficiency
def eval_step(state, batch):
    image, label = batch
    logits = state.apply_fn({"params": state.params}, label)
    b, t, c = logits.shape
    logits = logits.reshape((b * t, c))
    labels = labels.reshape((b * t))
    labels_one_hot = nn.one_hot(labels, num_classes=tokenizer.get_vocab_size())

    loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
    return loss


def train(state, num_epochs=100):
    for epoch in tqdm(range(num_epochs)):
        train_batch, labels_batch = batch_loader.get_batch(
            key, batch_size, context_lenght, is_train=True
        )

        batch = (train_batch, labels_batch)

        
        epoch_loss = jnp.array([])
        epoch_acc = jnp.array([])

        # for batch in batches:
        state, loss = _train_step(state, batch)

        jnp.append(epoch_loss, loss)
        # epoch_acc.append(acc)
        # We could use the loss and accuracy for logging here, e.g. in TensorBoard

        print(f"Loss at epoch {epoch}: {loss}")
        # print(f"Accuracy at epoch {epoch}: {epoch_acc.mean()}")
    return state

In [88]:
# Model init

data = jnp.ones(
    (batch_size, context_lenght), dtype=jnp.int32
)  # Example shape (batch_size, sequence_length)
labels = jnp.ones((batch_size, context_lenght), dtype=jnp.int32)

model = BenchmarkModel(vocab_size=tokenizer.get_vocab_size())

key, subkey = jax.random.split(key)
variables = model.init(rngs=subkey, data=data, labels=labels)

In [89]:
# Training

model_state = train_state.TrainState.create(
    apply_fn=model.apply,
    params=variables["params"],
    tx=optimizer,
)

trained_model_state = train(model_state, num_epochs=100)

  0%|          | 0/100 [00:00<?, ?it/s]

(4, 8)
Loss at epoch 0: 5.079814910888672
(4, 8)
Loss at epoch 1: 5.060153007507324
(4, 8)
Loss at epoch 2: 5.021048069000244
(4, 8)


  4%|▍         | 4/100 [00:00<00:02, 36.10it/s]

Loss at epoch 3: 4.962540626525879
(4, 8)
Loss at epoch 4: 4.88470458984375
(4, 8)
Loss at epoch 5: 4.787644386291504
(4, 8)
Loss at epoch 6: 4.671515464782715
(4, 8)
Loss at epoch 7: 4.536538124084473
(4, 8)


 10%|█         | 10/100 [00:00<00:01, 45.06it/s]

...

Loss at epoch 86: 0.8477829694747925
(4, 8)
Loss at epoch 87: 0.8477702140808105
(4, 8)
Loss at epoch 88: 0.8477793335914612
(4, 8)
Loss at epoch 89: 0.8477969169616699
(4, 8)
Loss at epoch 90: 0.8478058576583862
(4, 8)
Loss at epoch 91: 0.847798228263855
(4, 8)
Loss at epoch 92: 0.8477798700332642
(4, 8)


 96%|█████████▌| 96/100 [00:01<00:00, 61.14it/s]

Loss at epoch 93: 0.8477641344070435
(4, 8)
Loss at epoch 94: 0.8477603793144226
(4, 8)
Loss at epoch 95: 0.8477678894996643
(4, 8)
Loss at epoch 96: 0.8477767109870911
(4, 8)
Loss at epoch 97: 0.8477775454521179
(4, 8)
Loss at epoch 98: 0.8477693200111389
(4, 8)


100%|██████████| 100/100 [00:01<00:00, 57.94it/s]

Loss at epoch 99: 0.8477587699890137


In [90]:

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=trained_model_state.params,
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)


ή"Σαυ§ή.μ7f; öξêKNgnendar’statatatatatard owβνE4ὀb﻿fB}gndar’sepresend owê  νN
7Ær’stantandardaneprd owληüàùαzἆὖW;Œἰξäv.τr’s s statuerd stuendanepr’stantues seres owvἑ0”cHὀαςqmûndantueprenend owὸὸéG,
ŒYὰ(ëùἑœTêPi)û3εJgne,
D
ρ!;"CἄδæEἀτύü( υβiἄταâæ73βK5äδZM﻿ῢἄæiσ”ά
v/–ςgntatuentarene,
,
bιkη’statanere


In [91]:
class SingleAttentionHead():
    '''
    One attention head
    '''

In [92]:
class MultiHeadAttention():
    '''
    Multiple attention heads combined together
    '''

In [93]:
class FeedForward():
    '''A feed forward network'''